In [4]:
# Preprocessing script (run once)
import sqlite3
import pandas as pd

# Load the raw dataset
df_lstat = pd.read_excel("datasets/Ladesaeulenregister_SEP.xlsx", header=10)

df_lstat.head()

,Betreiber,Anzeigename (Karte),Straße,Hausnummer,Adresszusatz,PLZ,Ort,Kreis/kreisfreie Stadt,Bundesland,Breitengrad,...,Public Key3,Steckertypen4,P4 [kW],Public Key4,Steckertypen5,P5 [kW],Public Key5,Steckertypen6,P6 [kW],Public Key6
0,Albwerk GmbH & Co. KG,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,442398",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,smopi® - Multi Chargepoint Solution GmbH,smopi,Albstraße,15,NaN,72535,Heroldstatt,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,449424",...,NaN,AC Typ 2 Steckdose,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,smopi® - Multi Chargepoint Solution GmbH,smopi,Albstraße,15,NaN,72535,Heroldstatt,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,449424",...,NaN,AC Typ 2 Steckdose,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,smopi® - Multi Chargepoint Solution GmbH,smopi,Albstraße,15,NaN,72535,Heroldstatt,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,449424",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albwerk GmbH & Co. KG,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,0,NaN,72589,Westerheim,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,5105",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Clean and filter the data
df_lstat['Ort'] = df_lstat['Ort'].str.strip().str.lower()
df_lstat = df_lstat.loc[df_lstat['Ort'] == 'berlin']
df_lstat = df_lstat.dropna(subset=['Breitengrad', 'Längengrad'])
df_lstat['Breitengrad'] = df_lstat['Breitengrad'].str.replace(',', '.').astype(float)
df_lstat['Längengrad'] = df_lstat['Längengrad'].str.replace(',', '.').astype(float)
df_lstat = df_lstat[(df_lstat['Breitengrad'].between(-90, 90)) & (df_lstat['Längengrad'].between(-180, 180))]
df_lstat['Address'] = df_lstat['Straße'] + ' ' + df_lstat['Hausnummer'].astype(str)
df_lstat.head()

,Betreiber,Anzeigename (Karte),Straße,Hausnummer,Adresszusatz,PLZ,Ort,Kreis/kreisfreie Stadt,Bundesland,Breitengrad,...,Steckertypen4,P4 [kW],Public Key4,Steckertypen5,P5 [kW],Public Key5,Steckertypen6,P6 [kW],Public Key6,Address
28002,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,Leipziger Platz,19,NaN,1011,berlin,Kreisfreie Stadt Berlin,Berlin,52.510055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Leipziger Platz 19
28003,TotalEnergies Charging Solutions Deutschland GmbH,TotalEnergies Charging Solutions,Storkower Str.,175,T175-IT1-1322-212,1036,berlin,Kreisfreie Stadt Berlin,Berlin,52.527420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Storkower Str. 175
28004,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,Uhlandstr.,177,NaN,1062,berlin,Kreisfreie Stadt Berlin,Berlin,52.502775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uhlandstr. 177
28005,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,Soldiner Str.,63,NaN,1335,berlin,Kreisfreie Stadt Berlin,Berlin,52.560530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soldiner Str. 63
28006,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,Barmbecker Weg,1,NaN,1359,berlin,Kreisfreie Stadt Berlin,Berlin,52.533317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Barmbecker Weg 1


In [6]:

df_lstat = df_lstat.rename(columns={
    'Anzeigename (Karte)': 'Name',
    'Breitengrad': 'Latitude',
    'Längengrad': 'Longitude'
})
df_lstat['id'] = range(1, len(df_lstat) + 1)

# Save the processed data to a SQLite database
conn = sqlite3.connect('charging_stations.db')
df_lstat[['id', 'Name', 'Address', 'Latitude', 'Longitude', 'PLZ']].to_sql('stations', conn, if_exists='replace', index=False)
conn.close()

: 